In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
import statsmodels.api as sm
from patsy import dmatrices

C:\Users\chong\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
com1 = pd.read_csv('companies_cleaned.csv')

In [3]:
HFA = pd.read_csv('HFA 1994-2015.csv')
HFA = HFA.fillna(value = 0)
HFA['Gvkey'] = HFA['Gvkey'].apply(lambda x: str(x).replace(' ',''))
HFA['NCUSIP'] = HFA['NCUSIP'].apply(lambda x: str(x).replace(' ',''))
HFA.NCUSIP = HFA['NCUSIP'].apply(lambda x: x.zfill(9))
HFA['Gvkey'] = HFA['Gvkey'].astype(int)
def dat(x):
    if x['EventDate'] == 0:
        if x['Date13D'] == 0:
            return x['DateCross5pct']
        else:
            return x['Date13D']
    else:
        return x['EventDate']
    
#calculate the year of activism
HFA['date'] = HFA[['EventDate','Date13D','DateCross5pct']].apply(dat,axis = 1)
HFA['date'] = pd.to_datetime(HFA['date'].astype(int).astype(str),\
                             errors = 'coerce')

#define a variable 'year' according to variable 'date'
HFA['year'] = [i.year for i in HFA.date]
HFA = HFA.dropna(subset = ['year'])
HFA['year-1'] = HFA.year - 1

In [4]:
com1.shape

(214640, 43)

In [5]:
#calculate the skewness
cov = ['analyst','divyld','growth3yr','herfindahl','inst',\
 'levb','mv','q','rnd46','roa']
skewness = stats.skew(com1[cov].dropna(),axis = 0)

In [6]:
cov_rs = [cov[i] for i in range(len(cov)) if skewness[i] > 0]
print(cov_rs)
stats.skewtest(com1[cov_rs].dropna(),axis = 0)

['analyst', 'divyld', 'growth3yr', 'inst', 'levb', 'mv', 'q', 'rnd46']


SkewtestResult(statistic=array([ 130.77269161,  190.84413076,  189.7413617 ,   23.46541795,
        111.01601143,  205.39319992,  353.6276942 ,  234.36298934]), pvalue=array([  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
         9.20120568e-122,   0.00000000e+000,   0.00000000e+000,
         0.00000000e+000,   0.00000000e+000]))

In [7]:
columns = cov + ['yeara','gvkey','permno']
sam = com1[columns]
#cut the sample from 1993 to 2015
flag3 = sam.yeara > 1992
flag4 = sam.yeara <2015
sam = sam[flag3 & flag4]
sam.shape

(185964, 13)

In [38]:
sam['analyst'] = sam['analyst'].fillna(value = 0)
sam['herfindahl'] = sam['herfindahl'].fillna(value = 1)
sam['inst'] = sam['inst'].fillna(value = 0)
sam['divyld'] = sam['divyld'].fillna(value = 0)
sam1 = sam.copy()

In [11]:
pd.notnull(sam.rnd46).sum()/185964

0.43576713772558129

In [39]:
sam['rnd46'] = sam['rnd46'].fillna(value = 0)
sam[sam['rnd46']<0].rnd46 = 0
sam = sam.dropna(subset = cov, axis = 0)

In [40]:
sam.shape

(153018, 13)

In [43]:
sam[['q_log','mv_log']] = sam[['q','mv']].apply(np.log)
sam[['analyst_log','rnd46_log']] = \
sam[['analyst','rnd46']].apply(lambda x: np.log(1+x))

In [44]:
# auxiliary function
def au(x):
    m_y = HFA['year-1'] == x['yeara']
    m_g = HFA.Permno == x['permno']
    if len(HFA[m_y & m_g]) > 0:
        return 1
    else:
        return 0

In [13]:
sam['y'] = sam[['yeara','gvkey','permno']].apply(au,axis = 1)

In [14]:
y,X = dmatrices('y ~ analyst_log + divyld + growth3yr + \
herfindahl + inst + levb + mv_log + q_log + rnd46_log + roa',\
                data = sam,return_type = 'dataframe')

In [20]:
probit_model = sm.Probit(y,X)
probit_res = probit_model.fit()
probit_res.summary()

Optimization terminated successfully.
         Current function value: 0.094691
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:               153007
Model:                         Probit   Df Residuals:                   152995
Method:                           MLE   Df Model:                           11
Date:                Tue, 10 Oct 2017   Pseudo R-squ.:                 0.04280
Time:                        14:18:50   Log-Likelihood:                -14488.
converged:                       True   LL-Null:                       -15136.
                                        LLR p-value:                3.990e-271
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -2.0801      0.038    -55.095      0.000      -2.154      -2.006
analyst_log    -0.1693      0.012    -14.370      0.000      -0.192      -0.146
cash1_log       0.0114      0.006      1.805      0.071      -0.001       0.024
divyld         -0.0258      0.140     -0.184      0.854      -0.300       0.249
growth3yr      -0.0325      0.015     -2.174      0.030      -0.062      -0.003
herfindahl     -0.0667      0.030     -2.190      0.028      -0.126      -0.007
inst            1.0193      0.036     28.047      0.000       0.948       1.090
levb           -0.0318      0.031     -1.029      0.304      -0.092       0.029
mv_log         -0.0047      0.006     -0.722      0.470      -0.017       0.008
q_log          -0.1401      0.012    -11.508      0.000      -0.164      -0.116
rnd46_log       0.0306      0.005      6.124      0.000       0.021       0.040
roa             0.0123      0.019      0.654      0.513      -0.025       0.049
===============================================================================
"""

In [45]:
pd.isnull(sam1['rnd46']).sum()

104927

In [46]:
com1.shape

(214640, 43)

In [102]:
df1 = pd.pivot_table(com1,index="gvkey",values='rnd46',aggfunc=lambda x:pd.notnull(x).sum())
df2 = df1[df1.rnd46>0].index

In [103]:
df3 = com1[com1['gvkey'].isin(df2) | pd.isnull(com1['gvkey'])]
df3.shape

(106762, 43)

In [104]:
df3.rnd46 = df3.rnd46.fillna(value=0)
df3['analyst'] = df3['analyst'].fillna(value = 0)
df3['herfindahl'] = df3['herfindahl'].fillna(value = 1)
df3['inst'] = df3['inst'].fillna(value = 0)
df3['divyld'] = df3['divyld'].fillna(value = 0)
flag5 = df3.yeara > 1992
flag6 = df3.yeara <2015
df4 = df3[flag5 & flag6]
df4.shape

(94071, 43)

In [105]:
df4.dropna(subset=['gvkey']).shape

(94071, 43)

In [106]:
df2

Int64Index([  1013,   1017,   1021,   1034,   1036,   1037,   1043,   1050,
              1054,   1055,
            ...
            308992, 311524, 312079, 314866, 315318, 315629, 315639, 316056,
            317427, 318815],
           dtype='int64', name='gvkey', length=10552)

In [107]:
# pd.isnull(sam.permno)

In [108]:
# for i in sam1['permno'].dropna().values:
#     if pd.notnull(sam1[sam1['permno'] == i]['rnd46']).sum() >= 1:
#         sam1[sam1['permno'] == i].rnd46 = sam1[sam1['permno'] == i].rnd46.fillna(value = 0)

In [109]:
df4[['mv_log']] = df4[['mv']].apply(np.log)
df4[['analyst_log','rnd46_log']] = \
df4[['analyst','rnd46']].apply(lambda x: np.log(1+x))

In [110]:
df4['y'] = df4[['yeara','gvkey','permno']].apply(au,axis = 1)

In [112]:
y,X = dmatrices('y ~ analyst_log + divyld + growth3yr + \
herfindahl + inst + levb + mv_log + q + rnd46_log + roa',\
                data = df4,return_type = 'dataframe')
probit_model = sm.Probit(y,X)
probit_res = probit_model.fit()
probit_res.summary()

Optimization terminated successfully.
         Current function value: 0.098983
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                87611
Model:                         Probit   Df Residuals:                    87600
Method:                           MLE   Df Model:                           10
Date:                Wed, 11 Oct 2017   Pseudo R-squ.:                 0.04594
Time:                        20:58:18   Log-Likelihood:                -8672.0
converged:                       True   LL-Null:                       -9089.6
                                        LLR p-value:                5.681e-173
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -2.0325      0.048    -41.961      0.000      -2.127      -1.938
analyst_log    -0.1530      0.016     -9.658      0.000      -0.184      -0.122
divyld          0.0925      0.188      0.491      0.624      -0.277       0.462
growth3yr      -0.0360      0.018     -1.946      0.052      -0.072       0.000
herfindahl     -0.0633      0.039     -1.630      0.103      -0.139       0.013
inst            0.9854      0.048     20.514      0.000       0.891       1.080
levb           -0.1574      0.041     -3.879      0.000      -0.237      -0.078
mv_log          0.0040      0.006      0.616      0.538      -0.009       0.017
q              -0.0308      0.004     -8.460      0.000      -0.038      -0.024
rnd46_log       0.0144      0.007      2.196      0.028       0.002       0.027
roa            -0.0594      0.022     -2.642      0.008      -0.103      -0.015
===============================================================================
"""